In [0]:
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA default")

display(spark.sql("SELECT current_catalog(), current_schema()"))


In [0]:
%sql
SELECT 
  dataset_split,
  COUNT(*) AS total_snapshots,
  COUNT(DISTINCT vehicle_id) AS total_veiculos,
  MIN(time_step) AS min_time_step,
  MAX(time_step) AS max_time_step
FROM gold_fato_snapshot
GROUP BY dataset_split
ORDER BY dataset_split;


In [0]:
%sql
SELECT
  dataset_split,
  SUM(CASE WHEN vehicle_id IS NULL THEN 1 ELSE 0 END) AS nulos_vehicle_id,
  SUM(CASE WHEN time_step IS NULL THEN 1 ELSE 0 END)   AS nulos_time_step,
  SUM(CASE WHEN dataset_split IS NULL THEN 1 ELSE 0 END) AS nulos_split,
  SUM(CASE WHEN class_label IS NULL THEN 1 ELSE 0 END) AS nulos_class_label
FROM gold_fato_snapshot
GROUP BY dataset_split;


## Relatório automático de nulos das features

In [0]:
import pyspark.sql.functions as F

df = spark.table("gold_fato_snapshot")

# Features = tudo menos chaves e colunas de controle
excluded = {"vehicle_id", "time_step", "dataset_split", "class_label"}
feature_cols = [c for c in df.columns if c not in excluded]

# Calcula nulos por coluna
null_exprs = [F.sum(F.col(c).isNull().cast("int")).alias(c) for c in feature_cols]
nulls_df = df.select(*null_exprs)

# Transforma para formato longo (coluna, qtd_nulos)
nulls_pd = (
    nulls_df.toPandas()
    .T
    .reset_index()
    .rename(columns={"index": "coluna", 0: "qtd_nulos"})
)

total_rows = df.count()
nulls_pd["pct_nulos"] = (nulls_pd["qtd_nulos"] / total_rows) * 100
nulls_pd = nulls_pd.sort_values("pct_nulos", ascending=False)

nulls_pd.head(20)


In [0]:
#  salvar um CSV do relatório de nulos no próprio ambiente
nulls_pd.to_csv("/tmp/scanias_null_report.csv", index=False)


### Detectar colunas quase constantes

In [0]:

sample_df = df.select(feature_cols).sample(False, 0.05, seed=42)

# conta valores distintos por coluna (aproximação)
distinct_counts = []
for c in feature_cols:
    dc = sample_df.select(c).distinct().count()
    distinct_counts.append((c, dc))

distinct_counts_sorted = sorted(distinct_counts, key=lambda x: x[1])
distinct_counts_sorted[:20]

### Distribuição do tempo de estudo e reparo

In [0]:
%sql
SELECT 
  in_study_repair,
  COUNT(*) AS qtd_veiculos,
  AVG(length_of_study_time_step) AS media_tempo_estudo,
  MIN(length_of_study_time_step) AS min_tempo_estudo,
  MAX(length_of_study_time_step) AS max_tempo_estudo
FROM gold_fato_tempo_ate_evento
GROUP BY in_study_repair
ORDER BY in_study_repair;

TODO

barras comparando qtd_veiculos por in_study_repair.

### Taxa de reparo por especificação

#### - spec_0

In [0]:
%sql
SELECT 
  v.spec_0,
  COUNT(*) AS qtd_veiculos,
  AVG(CASE WHEN v.in_study_repair = 1 THEN 1 ELSE 0 END) AS taxa_reparo
FROM gold_dim_veiculo v
GROUP BY v.spec_0
ORDER BY taxa_reparo DESC;


#### spec_1

In [0]:
%sql
SELECT 
  v.spec_1,
  COUNT(*) AS qtd_veiculos,
  AVG(CASE WHEN v.in_study_repair = 1 THEN 1 ELSE 0 END) AS taxa_reparo
FROM gold_dim_veiculo v
GROUP BY v.spec_1
ORDER BY taxa_reparo DESC;


#### - spec_2


In [0]:
%sql
SELECT 
  v.spec_2,
  COUNT(*) AS qtd_veiculos,
  AVG(CASE WHEN v.in_study_repair = 1 THEN 1 ELSE 0 END) AS taxa_reparo
FROM gold_dim_veiculo v
GROUP BY v.spec_2
ORDER BY taxa_reparo DESC;


### Tempo observado médio por specs

In [0]:
%sql
SELECT 
  v.spec_0,
  AVG(v.length_of_study_time_step) AS media_tempo_estudo
FROM gold_dim_veiculo v
GROUP BY v.spec_0
ORDER BY media_tempo_estudo ASC;

### istribuição das classes 0–4

In [0]:
%sql
SELECT 
  dataset_split,
  class_label,
  COUNT(*) AS qtd
FROM gold_fato_snapshot
WHERE class_label IS NOT NULL
GROUP BY dataset_split, class_label
ORDER BY dataset_split, class_label;


### Comparar médias de features por classe

In [0]:
df = spark.table("gold_fato_snapshot").filter("class_label IS NOT NULL")

excluded = {"vehicle_id", "time_step", "dataset_split", "class_label"}
feature_cols = [c for c in df.columns if c not in excluded]

selected = feature_cols[:10]  # você pode trocar por uma lista manual depois
selected

In [0]:
agg_exprs = [F.avg(F.col(c)).alias(f"avg_{c}") for c in selected]

df_by_class = (
    df.groupBy("class_label")
      .agg(*agg_exprs)
      .orderBy("class_label")
)

display(df_by_class)

### Snapshot X Specs

In [0]:
%sql
SELECT
  v.spec_0,
  f.class_label,
  AVG(f.666_0) AS media_666_0
FROM gold_fato_snapshot f
JOIN gold_dim_veiculo v
  ON f.vehicle_id = v.vehicle_id
WHERE f.class_label IS NOT NULL
GROUP BY v.spec_0, f.class_label
ORDER BY v.spec_0, f.class_label;
